In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/train.csv')
labels = train_data['label'] 
train_data = train_data.drop('label', axis = 1)

In [ ]:
import torch

data = torch.from_numpy(train_data.values.reshape(-1,1,28,28))
data = data.type(torch.float32)
labels = torch.from_numpy(labels.values)
labels = labels.type(torch.long)

In [ ]:
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset

train_data = TensorDataset(data, labels)

idx = list(range(len(train_data)))
mid = int(0.99*len(train_data))

# split and suffle indexes
train_idx = SubsetRandomSampler(idx[:mid])
test_idx = SubsetRandomSampler(idx[mid:])

# data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=20, sampler=train_idx)
test_loader = torch.utils.data.DataLoader(train_data, batch_size=20, sampler=test_idx)


In [ ]:
print(len(train_loader)*20)
print(len(test_loader)*20)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()        
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.2)
    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)   
        x = F.log_softmax(self.fc3(x), dim=1)
        return x
model = Net()
print(model)

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.001)

In [ ]:
# number of epochs 
epochs = 85

model.train() # prep model for training

for epoch in range(epochs):
    
    train_loss = 0.0
    total = 0.0
    
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        total += len(data) 
        
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/total
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1,train_loss))
    

In [ ]:
correct = 0.0
test_loss = 0.0
total= 0.0

model.eval() # prep model for evaluation

for data, target in test_loader:
    output = model(data)
    loss = criterion(output, target)  
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)
    correct += np.squeeze(pred.eq(target.data.view_as(pred))).sum()
    total += len(data)
    
# calculate and print average test loss
test_loss = test_loss/total
print('Test Loss: {:.6f}'.format(test_loss)) 

# calculate and print accuracy
print('Test Accuracy: {:.2f}% ({}/{})' .format(100. * np.float(correct) / total, 
                                           correct, total))
        


In [ ]:
test_data = pd.read_csv('../input/test.csv')
test_data = torch.from_numpy(test_data.values.reshape(-1,1,28,28))
test_data = test_data.type(torch.float)

In [ ]:
model.eval() 
result = []

for data in test_data:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    _, pred = torch.max(output, 1)
    pred = pred.numpy()
    result += pred.tolist()

results = pd.Series(result,name="Label")
Imageld = pd.Series(range(1,len(result)+1),name="Imageld")

submission = pd.concat([Imageld,results],axis = 1)
submission.to_csv("submission.csv",index=False)

In [ ]:
x = pd.read_csv('submission.csv')
print(x[:10])